In [1]:
import pandas as pd
import peakutils
import matplotlib.pyplot as plt
import numpy as np
import datetime

In [2]:
# create giant data object, to house all data
data = {}

In [3]:
# read all the labels for the data into a df
def read_labels_for_all_houses():
    houses = []
    for house in range(1, 7):
        hi = 'Data/low_freq/house_{}/labels.dat'.format(house)
        df = pd.read_csv(hi, sep=" ", header=None, names = ["appliance_id","appliance_name"], dtype={"appliance_id":"int64","appliance_name":"string"})
        df['house_id'] = house
        houses.append(df)
    return pd.concat(houses).reset_index(drop=True)

labels = read_labels_for_all_houses()
for house in range(1,7):
    print('House {}: \n'.format(house), labels[labels['house_id'] == house] , '\n')

data['labels_for_houses'] = labels

House 1: 
     appliance_id   appliance_name  house_id
0              1            mains         1
1              2            mains         1
2              3             oven         1
3              4             oven         1
4              5     refrigerator         1
5              6        dishwaser         1
6              7  kitchen_outlets         1
7              8  kitchen_outlets         1
8              9         lighting         1
9             10     washer_dryer         1
10            11        microwave         1
11            12     bathroom_gfi         1
12            13    electric_heat         1
13            14            stove         1
14            15  kitchen_outlets         1
15            16  kitchen_outlets         1
16            17         lighting         1
17            18         lighting         1
18            19     washer_dryer         1
19            20     washer_dryer         1 

House 2: 
     appliance_id   appliance_name  house_id
20      

In [4]:
# read all the appliance data points for all houses
def read_channel_data(labels_for_houses, house_id):
    house_labels = labels_for_houses[labels_for_houses['house_id'] == house_id]
    # print(house_labels)
    df = pd.DataFrame()
    for index,row in house_labels.reset_index().iterrows():
        path = 'data/low_freq/house_{}/'.format(house_id)
        file = path + 'channel_{}.dat'.format(row['appliance_id'])
        cname = (row['house_id'], row['appliance_id'], row['appliance_name'])
        file_content = pd.read_table(file, sep = ' ', names = ['unix_time', cname], 
                                        dtype = {'unix_time': 'int64', cname:'float64'})
        if(index==0):
            df = file_content
        else:
            df = pd.merge(df, file_content, how='inner', on='unix_time')
    
    df['timestamp'] = df['unix_time'].astype("datetime64[s]")
    df = df.set_index(df['timestamp'].values)
    df.drop(['unix_time','timestamp'], axis=1, inplace=True)
    df.columns = pd.MultiIndex.from_tuples(df.columns, names=["house_id","appliance_id", "appliance_name"])
    return df

# print(data['labels_for_houses'])
data['channels'] = {}
dfs = []
for house in range(1,7):
    print('reading channels for house {}'.format(house))
    # data['channels'][house] = read_channel_data(data['labels_for_houses'], house)
    df = read_channel_data(data['labels_for_houses'], house)
    # print(df.head())
    dfs.append(df)

appliance_data = pd.concat(dfs, axis=1)
print(appliance_data.columns)
print(appliance_data.head())

reading channels for house 1
reading channels for house 2
reading channels for house 3
reading channels for house 4
reading channels for house 5
reading channels for house 6
MultiIndex([(1,  1,            'mains'),
            (1,  2,            'mains'),
            (1,  3,             'oven'),
            (1,  4,             'oven'),
            (1,  5,     'refrigerator'),
            (1,  6,        'dishwaser'),
            (1,  7,  'kitchen_outlets'),
            (1,  8,  'kitchen_outlets'),
            (1,  9,         'lighting'),
            (1, 10,     'washer_dryer'),
            ...
            (6,  8,     'refrigerator'),
            (6,  9,        'dishwaser'),
            (6, 10,  'outlets_unknown'),
            (6, 11,  'outlets_unknown'),
            (6, 12,    'electric_heat'),
            (6, 13,  'kitchen_outlets'),
            (6, 14,         'lighting'),
            (6, 15, 'air_conditioning'),
            (6, 16, 'air_conditioning'),
            (6, 17, 'air_condit

In [ ]:
# example of indexing with multiindex columns


# print columns
print(appliance_data.columns)

# print only names
print(appliance_data.columns.get_level_values('appliance_name'))

# print only appliance ids
print(appliance_data.columns.get_level_values('appliance_id'))


# print only ids
print(appliance_data.columns.get_level_values('house_id'))

# index using appliance name 
print(appliance_data.xs('mains',level='appliance_name', axis=1).head())

# and using appliance id
print(appliance_data.xs(2,level='appliance_id', axis=1).head())

# index column based on position
print(appliance_data.iloc[:,1].head())


In [ ]:
# extract all mains
def hours(time):
    return time.hour + time.minute/60 + time.second / (60*60)

def index_to_hours(df):
    return [hours(t) for t in df.index.time]

In [ ]:
mainss = appliance_data.xs("mains", level="appliance_name", axis=1)
timeToInt = index_to_hours(mainss)
mainss['date'] = mainss.index.date

# plot 
# for g,o in mainss.groupby('date'):
#     o = o.iloc[:,:-1]
#     # print(index_to_hours(o))
#     for i in range(len(o.columns)):
#         plt.plot(index_to_hours(o), o.iloc[:,i], label=o.columns[i])
#     plt.title(g)
#     plt.legend(loc="upper right")
#     plt.xlim([0,24])
#     plt.show()

plots = {}
fig = plt.figure()
fig.set_size_inches(20,4* (len(mainss.columns)-1))
# fig.set_size_inches(20)
for c in range(len(mainss.columns)-1):
    plots[c] = fig.add_subplot(len(mainss.columns)-1,1, c+1)

for g,o in mainss.groupby('date'):
    o = o.iloc[:,:-1]
    amount = len(o.columns)
    for i in range(amount):
        plots[i].fill_between(index_to_hours(o), o.iloc[:,i], alpha=(1/amount/5), color="blue")
        print(o.iloc[:,1].loc[datetime.datetime.fromtimestamp(1306016892) - datetime.timedelta(30):datetime.datetime.fromtimestamp(1306016892) + datetime.timedelta(30)])
        # plots[i].plot(index_to_hours(o), o.iloc[:,i], label=g, color = "blue", alpha = 0.)


for i in range(len(mainss.columns[:-1])):
    plots[i].set(xlim=[0,24], title=mainss.columns[i])
    # plots[i].legend(loc="upper right")
    # plots[i].show()`
plt.show()


In [ ]:
# resample signals
cleaned = appliance_data.resample("3s").median()
cleaned = cleaned.interpolate(method="pad", limit=5)

idx = cleaned.index

channels = {}
# channels contains lists of continuous 
for c in cleaned.columns:
    series = cleaned.loc[:,c]
    series = series.rolling(8, min_periods=1).median()
    # print(series)
    h,a,_ = c
    # groups based on wrap by nan
    x = (series.shift(1).isnull() & series.notnull()).cumsum()
    print(x)
    # add group if at least one hour of data is available
    channels[c] = [d.dropna() for _, d in series.groupby(x) ]

print(channels)



In [5]:
def onehouse(df, i):
    house = df.xs(i, level='house_id', axis=1)

    # get the index of the first and last non-null value in each column
    first_non_null = house.apply(lambda x: x.first_valid_index()).values[0]
    last_non_null = house.apply(lambda x: x.last_valid_index()).values[0]
    
    # slice the DataFrame to keep only the rows between the first and last non-null values
    house = house.loc[first_non_null:last_non_null]

    return house

In [6]:
a=onehouse(appliance_data,1)
a.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 999148 entries, 2011-04-18 13:22:13 to 2011-05-24 19:56:34
Data columns (total 20 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   (1, mains)             406748 non-null  float64
 1   (2, mains)             406748 non-null  float64
 2   (3, oven)              406748 non-null  float64
 3   (4, oven)              406748 non-null  float64
 4   (5, refrigerator)      406748 non-null  float64
 5   (6, dishwaser)         406748 non-null  float64
 6   (7, kitchen_outlets)   406748 non-null  float64
 7   (8, kitchen_outlets)   406748 non-null  float64
 8   (9, lighting)          406748 non-null  float64
 9   (10, washer_dryer)     406748 non-null  float64
 10  (11, microwave)        406748 non-null  float64
 11  (12, bathroom_gfi)     406748 non-null  float64
 12  (13, electric_heat)    406748 non-null  float64
 13  (14, stove)            406748 non-null  float64
 14  (1

In [ ]:
for a in range(1, 7):
    df = onehouse(cleaned, a)
    df.columns = df.columns.droplevel(0) # Drop first level of MultiIndex
    df = df.reset_index(drop=False)
    new_cols = [f"{i}_{col}" for i, col in enumerate(df.columns)] # Create new column names
    df.columns = new_cols # Assign new column names
    df.to_feather("Data/cleaned/house/house_{0}.feather".format(a))


In [ ]:
count=0
for c in [c for c in channels if c[2] == "mains"]:
    for y in channels[c]:
        try:
            y.index[0],y.index[-1]
            count=count+1
        except IndexError:
            print(y)
            print(c)

print(y.index[0])
print(c)
print(count)

In [ ]:
# perform fft on signals

for c in [c for c in channels if c[2] == "mains"]:
    for y in channels[c]:
        try:
            y.index[0],y.index[-1]
            count=count+1
        except IndexError:
            pass
        else:
            x = index_to_hours(y)
            N = len(x)
            T = 1200 # samples per hour 
            fig = plt.figure()
            fig.set_size_inches(10,5)
            ax1 = fig.add_subplot(1,2,1) # signal
            ax2 = fig.add_subplot(1,2,2) # freqs
            ax1.plot(x,y)
            ax1.set_title("{} - from {} to {}".format(c,y.index[0],y.index[-1]))
            # plt.show()   
            yf = np.fft.fft(y)[:N//2] * 2./N
            v1 = abs(yf[:N//2]) * 2./N
            xf = np.fft.fftfreq(N, d=1/T)[0:N//2]
            peaks = peakutils.indexes(v1, thres=0.2, min_dist=2)
            params = []
            for p in peaks:
                freq, ampl, phase = par = (xf[p], v1[p], np.angle(yf[p]))
                params.append(par)
                print("Peak at:", freq, "\tMagnitude:", ampl, "\tPhase:", phase/ np.pi, 'PI')
                ax2.scatter(xf[p], v1[p], s=10, marker='s')
                ax2.annotate("F:{:.2f} M:{:.2f} P:{:.2f}".format(*par), (freq,ampl))
            # reconstruct
            signal = np.array([0. for i in range(N)])
            # print(signal    )
            for (_f,_a,_p) in params:
                # signal += np.sin(x)
                signal += _a* N * np.sin(_f*np.array(x) * 2 * np.pi + _p)
            # print(signal.max())
            # print(peaks)
            ax1.plot(x,signal, alpha=0.3)
            ax2.plot(xf,v1)
            ax2.set_xlim([0,10])
            plt.show()



In [ ]:
plot = False
_print = False

In [ ]:
# extract diffs for signal 
all_diffs_time = {}
all_diffs_house = {c: None for c in channels}


for c in [c for c in channels if c[2] != "mains"]:
    print(c)
    arrs = []
    for y in channels[c]:
        G_LENGTH = 1200 if plot else len(y) # 6 * 200 * 3s = 1h 
        groups = [i // G_LENGTH for i in range(len(y))]
        for g, yg in y.groupby(groups):
            df = yg

            K_SIZE = 2
            diff = (df - df.shift(K_SIZE) +  df.shift(-K_SIZE) - df)
            peaks = peakutils.indexes(abs(diff), thres=0.1, min_dist=2)
            arrs.extend([(yg.index[p], diff.iloc[p]) for p in peaks])
            if(plot):
                # print(1/ diff.max())
                plt.plot(yg.index,df, label="Original")
                plt.title(c)
                plt.plot(yg.index,diff, label = "Difference")
                plt.legend()
                plt.show()
    vals = [i[1] for i in arrs]
    index = [i[0] for i in arrs]
    all_diffs_house[c] = pd.Series(vals, index=index)

In [ ]:
# filter relevant diffs 
all_diffs_house_cleaned = {}
for c, d in all_diffs_house.items():
    if(not d is None and not d.empty):
        print(c)
        abs_d = abs(d)
        max_d = max(abs_d)
        # filter out if less then 15 w
        trueth = (abs_d > 10.) & (abs_d > 0.05 * max_d)
        res = d[trueth]
        others = d[~trueth]
        all_diffs_house_cleaned[c] = { "d": res, "low_t": abs(others).max(),"high_t" : abs(res).min()}
        print("\t\t IN \t/ OUT")
        print("MIN \ MAX\t", abs(res).min(), '/', abs(others).max())
        print("COUNT\t\t", len(res), '\t/', len(d))
        print()
print(all_diffs_house_cleaned)

In [ ]:
current_house = 1
for c, dobj in all_diffs_house_cleaned.items():
    print(c)
    og = pd.concat(channels[c])
    channel = all_diffs_house_cleaned[c]
    ser = og.rolling(5).median().fillna(0)
    ser_prev = ser.shift(1).fillna(0)
    upper_t = ser >= channel["high_t"]
    upper_t_prev = ser_prev >= channel["high_t"]
    lower_t = ser <= channel["low_t"]
    # lower_t_prev = ser_prev <= channel["low_t"]
    schmitt = upper_t | (lower_t & upper_t_prev)
    begin_edge = schmitt & (~schmitt.shift(1).fillna(False))
    labels = pd.Series(index = og.index, data=np.repeat(0, len(og)))
    labels[begin_edge] = 1
    labels[schmitt] = labels[schmitt].cumsum() 

    stats = {
        "start" :[],
        "mean" :[],
        "len" :[],
        "var" :[],
        "max" :[],
        "min" :[]
    }
    cl_df = pd.Series(index=og.index)
    gated = pd.Series(index=og.index)
    means = [] 
    lens = []
    for (g,d) in ser.groupby(labels.iloc[1:]):
        t_span = (d.index[-1] - d.index[0]).total_seconds()
        samples_length_seconds = (len(d) + 10)*3 * 1.1 #10 error samples and deviation of 1.1 for allowance
        if(t_span > samples_length_seconds):
            print(t_span, samples_length_seconds)
            print("pass", g, d.mean(), len(d))
            continue
        if(plot):
            gated[d.index] = d.mean()
            means.append(d.mean())
            lens.append(len(d))
        cl_df.loc[d.index] = d.mean()
        stats["start"].append(d.index[0])
        stats["mean"].append( d.mean())
        stats["len"].append((d.index[-1]-d.index[0]).total_seconds())
        stats["var"].append( d.var() / d.mean())
        stats["max"].append( d.max())
        stats["min"].append( d.min())
    cl_df = cl_df[~cl_df.isna()]
    stats_df = pd.DataFrame(data=stats, columns= stats.keys())
    stats_df.to_csv('Data/cleaned/stats/{0}_{1}_{2}.csv'.format(c[0],c[1],c[2]), index = False, header=True, encoding='utf-8')
    labels.to_csv('Data/cleaned/labels/{0}_{1}_{2}.csv'.format(c[0],c[1],c[2]), index = True, header=True, encoding='utf-8')
    cl_df.to_csv('Data/cleaned/time/{0}_{1}_{2}.csv'.format(c[0],c[1],c[2]), index = True, header=False, encoding='utf-8')
    if(plot):
        plt.scatter(lens[1:], means[1:])
        plt.show()
        # plt.plot(ser)
        plt.scatter(gated.index,gated, s=.1, label="schmitt")
        plt.plot(og, alpha = .1, label="original")
        plt.legend()
        plt.show()
    if(c[0] != current_house):
        current_house+= 1
    # print(ser)

In [ ]:
diffs_with_mains_per_house = {}
index = pd.date_range(start= appliance_data.index[0], end = appliance_data.index[-1], freq='3S')
for house in range(1,7):
    diffs_with_mains_per_house[house] = pd.DataFrame(index=index)
    this_house = {c:d['d'] for c, d in all_diffs_house_cleaned.items() if c[0] == house}
    for c, d in this_house.items():
        diffs_with_mains_per_house[house] = pd.merge(diffs_with_mains_per_house[house],d.rename(c[1]), how='left', left_index=True, right_index=True)

    mains = appliance_data.xs('mains',level='appliance_name', axis=1).xs(house,level='house_id', axis=1)
    diffs_with_mains_per_house[house] = pd.merge(diffs_with_mains_per_house[house], mains, how='left', left_index=True, right_index=True).resample('3S').median()
    # plot against mains
    df = diffs_with_mains_per_house[house]
    pks = df.loc[~df.isnull().all(axis=1)]
    plt.figure(figsize=(300,10))
    plt.plot(mains)
    for i in range(pks.shape[1]):
        plt.scatter(pks.index, abs(pks.iloc[:,i]), s=2, c='red')
    plt.show()


In [ ]:
def show_value_spectrum(channel_data, use_log=False):
    channel_data = {k:v for k,v in channel_data.items()  if not v is None }
    f_loc = lambda k: ((k-1) // 3, (k-1) % 3)
    fig1, plots = plt.subplots(2,3)
    fig1.set_size_inches(30,12)
    fig2 = plt.figure()
    fig2.set_size_inches(15,12)
    ax = fig2.gca()

    # setup subplots
    keys = channel_data.keys()
    for i in range(1,7):
        (x,y) = f_loc(i)
        xtcks = [k[1] for k in keys if k[0]==i]
        xtck_labels = [k[2] for k in keys if k[0]==i]
        plots[x][y].set_xticks(xtcks)
        plots[x][y].set_xticklabels(xtck_labels, rotation=60)

    # setup main plot
    unique_keys = list({k[2] for k in keys})
    unique_keys = {unique_keys[i]: i for i in range(len(unique_keys))}
    colors = "bgrcmy"
    ax.set_xticks(range(len(unique_keys)))
    ax.set_xticklabels(unique_keys.keys(), rotation=60)


    for channel_key, data in channel_data.items():    
        if(len(data) == 0):
            continue
        # if(not data is None):
        if use_log:
            data = np.log10(data)
        (x,y) = f_loc(channel_key[0])
        keys = channel_key[1] + np.random.uniform(-.35,.35, size = len(data))
        unique_key = unique_keys[channel_key[2]] 

        keys_1 = channel_key[1] + np.random.uniform(-.35,.35, size = len(data)) 
        keys_2 = unique_key + np.random.uniform(-.35,.35, size = len(data)) 
        mult = np.clip(np.log(500/len(data)),1, 5)
        plots[x][y].scatter(keys_1,data, s = .3 * mult, c = data, alpha = 0.15 * mult)
        ax.scatter(keys_2, data, s = .3 * mult, c = colors[channel_key[0]-1], alpha = 0.15 * mult)
    
    fig1.show()
    ax.legend()
    fig2.show()


diffs_object = {c:d['d'] for c, d in all_diffs_house_cleaned.items()}
show_value_spectrum(diffs_object, use_log=True)